<a href="https://colab.research.google.com/github/kylama/musicrecognition/blob/main/MaSridharMusicCategorizationKNNMultiple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_blobs

# Import the k-Nearest Neighbors model from the sklearn library
from sklearn.neighbors import KNeighborsClassifier

# Import the train/test data split function from the sklearn library
from sklearn.model_selection import train_test_split

In [13]:
df = pd.read_csv('songs_normalize.csv')

xDf = df[['danceability', 'energy', 'key', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]
yDf = df[['genre', 'song', 'artist']]

display(xDf)
display(yDf)

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.751,0.834,1,-5.444,0.0437,0.3000,0.000018,0.3550,0.894,95.053
1,0.434,0.897,0,-4.918,0.0488,0.0103,0.000000,0.6120,0.684,148.726
2,0.529,0.496,7,-9.007,0.0290,0.1730,0.000000,0.2510,0.278,136.859
3,0.551,0.913,0,-4.063,0.0466,0.0263,0.000013,0.3470,0.544,119.992
4,0.614,0.928,8,-4.806,0.0516,0.0408,0.001040,0.0845,0.879,172.656
...,...,...,...,...,...,...,...,...,...,...
1995,0.842,0.734,1,-5.065,0.0588,0.0427,0.000000,0.1060,0.952,137.958
1996,0.552,0.702,9,-5.707,0.1570,0.1170,0.000021,0.1050,0.564,169.994
1997,0.847,0.678,9,-8.635,0.1090,0.0669,0.000000,0.2740,0.811,97.984
1998,0.741,0.520,8,-7.513,0.0656,0.4500,0.000002,0.2220,0.347,102.998


,genre,song,artist
0,pop,Oops!...I Did It Again,Britney Spears
1,"rock, pop",All The Small Things,blink-182
2,"pop, country",Breathe,Faith Hill
3,"rock, metal",It's My Life,Bon Jovi
4,pop,Bye Bye Bye,*NSYNC
...,...,...,...
1995,pop,Sucker,Jonas Brothers
1996,pop,Cruel Summer,Taylor Swift
1997,"hip hop, country",The Git Up,Blanco Brown
1998,pop,Dancing With A Stranger (with Normani),Sam Smith


In [14]:
y_relationship = np.reshape(yDf['genre'], (len(yDf), ))
for i, value in enumerate(yDf['genre']):
  x = value.split(", ")
  value = x[0]
  y_relationship[i] = value
print(y_relationship[0:15])

['pop' 'rock' 'pop' 'rock' 'pop' 'hip hop' 'hip hop' 'pop' 'pop'
 'Dance/Electronic' 'pop' 'pop' 'pop' 'rock' 'pop']


In [15]:
# Function for transforming genre data (y) into numerical data

for i, element in enumerate(y_relationship):
  if element == 'pop':
    yDf['genre'][i] = 1
  elif element == 'hip hop':
    yDf['genre'][i] = 2
  elif element == 'rock':
    yDf['genre'][i] = 3
  elif element == 'country':
    yDf['genre'][i] = 4
  elif element == 'metal':
    yDf['genre'][i] = 5
  elif element == 'R&B':
    yDf['genre'][i] = 6
  elif element == 'Dance/Electronics':
    yDf['genre'][i] = 7
  elif element == 'Folk/Acoustic':
    yDf['genre'][i] = 8
  elif element == 'latin':
    yDf['genre'][i] = 9
  elif element == 'blues':
    yDf['genre'][i] = 10
  else:
    yDf['genre'][i] = 0
print(yDf)
print(yDf.shape)

<ipython-input-15-b0f29f85fcb7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yDf['genre'][i] = 1
<ipython-input-15-b0f29f85fcb7>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yDf['genre'][i] = 3
<ipython-input-15-b0f29f85fcb7>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yDf['genre'][i] = 2
<ipython-input-15-b0f29f85fcb7>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the 

     genre                                    song          artist
0        1                  Oops!...I Did It Again  Britney Spears
1        3                    All The Small Things       blink-182
2        1                                 Breathe      Faith Hill
3        3                            It's My Life        Bon Jovi
4        1                             Bye Bye Bye          *NSYNC
...    ...                                     ...             ...
1995     1                                  Sucker  Jonas Brothers
1996     1                            Cruel Summer    Taylor Swift
1997     2                              The Git Up    Blanco Brown
1998     1  Dancing With A Stranger (with Normani)       Sam Smith
1999     2                                 Circles     Post Malone

[2000 rows x 3 columns]
(2000, 3)


In [16]:
yDf

,genre,song,artist
0,1,Oops!...I Did It Again,Britney Spears
1,3,All The Small Things,blink-182
2,1,Breathe,Faith Hill
3,3,It's My Life,Bon Jovi
4,1,Bye Bye Bye,*NSYNC
...,...,...,...
1995,1,Sucker,Jonas Brothers
1996,1,Cruel Summer,Taylor Swift
1997,2,The Git Up,Blanco Brown
1998,1,Dancing With A Stranger (with Normani),Sam Smith


In [17]:
copy = pd.DataFrame(yDf)
copy['genre'].unique()

array([1, 3, 2, 0, 5, 6, 8, 9, 4], dtype=object)

In [18]:
copy

,genre,song,artist
0,1,Oops!...I Did It Again,Britney Spears
1,3,All The Small Things,blink-182
2,1,Breathe,Faith Hill
3,3,It's My Life,Bon Jovi
4,1,Bye Bye Bye,*NSYNC
...,...,...,...
1995,1,Sucker,Jonas Brothers
1996,1,Cruel Summer,Taylor Swift
1997,2,The Git Up,Blanco Brown
1998,1,Dancing With A Stranger (with Normani),Sam Smith


In [19]:
def KNN(k):
  accuracy = [] # declare an empty list to hold the accuracy
  for i in range(1):
    X_train, X_test, y_train, y_test, song_train, song_test, artist_train, artist_test = train_test_split(xDf, yDf['genre'], yDf['song'], yDf['artist'], test_size = 0.2, random_state = i) # split the data into train/test split
    y_train = np.array(y_train.astype(int))
    y_test = np.array(y_test.astype(int))
    song_train = np.array(song_train.astype(str))
    song_test = np.array(song_test.astype(str))
    artist_train = np.array(artist_train.astype(str))
    artist_test = np.array(artist_test.astype(str))

    scaler = StandardScaler() # standardize the data
    scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = KNeighborsClassifier(n_neighbors = k, weights = "distance") # train a KNN model using the training data and labels
    model.fit(X_train_scaled, y_train)
    print(X_train_scaled)
    print(y_relationship)

    y_hat = model.predict(X_test_scaled)

    score = model.score(X_test_scaled, y_test) # calculate the accuracy of the model using the testing data

    accuracy.append(score) # append the accuracy to the list containing all accuracies;

  print(accuracy)

  y_pred = model.predict(X_test)

  def num_to_genre(holder):
    if holder == 1:
      holder = 'pop'
    elif holder == 2:
      holder = 'hip hop'
    elif holder == 3:
      holder = 'rock'
    elif holder == 4:
      holder = 'country'
    elif holder == 5:
      holder = 'metal'
    elif holder == 6:
      holder = 'R&B'
    elif holder == 7:
      holder = 'Dance/Electronics'
    elif holder == 8:
      holder = 'Folk/Acoustic'
    elif holder == 9:
      holder = 'latin'
    elif holder == 10:
      holder = 'blues'
    else:
      holder = 'none'
    return holder

  for i in range(len(y_test)):
    true = y_test[i]
    pred = y_pred[i]
    song = song_test[i]
    artist = artist_test[i]
    print("Song Title: ", song)
    print("Artist: ", artist)
    print("True: ", num_to_genre(true))
    print("Pred: ", num_to_genre(pred))
    print("")

  avg_accuracy = sum(accuracy) / len(accuracy) # calculate the average accuracy by averaging all elements in the list holding all accuracies
  print("average accurcy of k =", k, ":", avg_accuracy)

In [20]:
KNN(25)

[[-1.36873076  0.93092666 -1.22940199 ... -0.49460835 -0.27534948
  -1.04150617]
 [ 0.96168177 -2.5052336   0.42473384 ... -0.72381416  1.34450304
   1.72229586]
 [-0.53438553  0.18050086  0.97611245 ...  0.83015917  0.64317025
   2.24356642]
 ...
 [-0.32579922  0.08176062  0.97611245 ... -0.25629038 -0.82284153
  -0.44961499]
 [ 1.2206165  -0.83981493 -0.40233408 ... -0.6649356  -0.20295383
  -1.11492771]
 [-0.08844239 -0.64233445 -0.67802338 ...  0.67595343  0.82868408
   0.15646753]]
[1 3 1 ... 2 1 2]
[0.64]
Song Title:  Left Outside Alone
Artist:  Anastacia
True:  pop
Pred:  pop

Song Title:  Tongue Tied
Artist:  Grouplove
True:  pop
Pred:  hip hop

Song Title:  Louder (feat. Sian Evans) - Radio Edit
Artist:  DJ Fresh
True:  pop
Pred:  pop

Song Title:  Last Night
Artist:  Keyshia Cole
True:  hip hop
Pred:  hip hop

Song Title:  Rake It Up (feat. Nicki Minaj)
Artist:  Yo Gotti
True:  hip hop
Pred:  hip hop

Song Title:  The Lazy Song
Artist:  Bruno Mars
True:  pop
Pred:  pop

Song 

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(



Artist:  Sophie Ellis-Bextor
True:  pop
Pred:  hip hop

Song Title:  Demons
Artist:  Imagine Dragons
True:  rock
Pred:  pop

Song Title:  What's Left Of Me
Artist:  Nick Lachey
True:  pop
Pred:  pop

Song Title:  Toxic
Artist:  Britney Spears
True:  pop
Pred:  pop

Song Title:  What Goes Around.../...Comes Around (Interlude)
Artist:  Justin Timberlake
True:  pop
Pred:  pop

Song Title:  Lollipop
Artist:  Lil Wayne
True:  hip hop
Pred:  hip hop

Song Title:  Don't Phunk With My Heart
Artist:  Black Eyed Peas
True:  hip hop
Pred:  pop

Song Title:  Nine Million Bicycles
Artist:  Katie Melua
True:  pop
Pred:  hip hop

Song Title:  Pop Style
Artist:  Drake
True:  hip hop
Pred:  pop

Song Title:  Take Me Back to London (feat. Stormzy)
Artist:  Ed Sheeran
True:  pop
Pred:  pop

Song Title:  Homemade Dynamite - REMIX
Artist:  Lorde
True:  pop
Pred:  hip hop

Song Title:  Everytime
Artist:  Britney Spears
True:  pop
Pred:  pop

Song Title:  King's Dead (with Kendrick Lamar, Future & James Bla